# TensorFlow张量与NumPy互操作

TensorFlow与NumPy的紧密集成是其设计的核心特性之一。本notebook详细探讨两者之间的数据转换、互操作注意事项及性能影响。

## 学习目标
1. 掌握TensorFlow张量与NumPy数组的双向转换
2. 理解默认数据类型差异及其影响
3. 了解数据转换的性能开销
4. 掌握在实际项目中的最佳实践

## 1. 环境设置

In [ ]:
import tensorflow as tf
import numpy as np
import time

# 设置随机种子
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"NumPy版本: {np.__version__}")

## 2. 数据类型转换基础

### 2.1 NumPy数组转TensorFlow张量

使用`tf.constant()`或`tf.convert_to_tensor()`将NumPy数组转换为TensorFlow张量。

In [ ]:
# NumPy数组转TensorFlow张量
np_array = np.array([[1, 2, 3], [4, 5, 6]])
print(f"NumPy数组:\n{np_array}")
print(f"NumPy dtype: {np_array.dtype}\n")

# 方法1: 使用tf.constant
tensor_from_constant = tf.constant(np_array)
print(f"tf.constant转换结果: {tensor_from_constant}")

# 方法2: 使用tf.convert_to_tensor
tensor_from_convert = tf.convert_to_tensor(np_array)
print(f"tf.convert_to_tensor转换结果: {tensor_from_convert}")

### 2.2 TensorFlow张量转NumPy数组

使用`.numpy()`方法或`np.array()`函数将张量转换回NumPy数组。

In [ ]:
# TensorFlow张量转NumPy数组
tensor = tf.constant([[1, 2, 3], [4, 5, 6]])
print(f"TensorFlow张量:\n{tensor}\n")

# 方法1: 使用.numpy()方法（推荐）
np_from_numpy_method = tensor.numpy()
print(f".numpy()方法结果:\n{np_from_numpy_method}")
print(f"类型: {type(np_from_numpy_method)}\n")

# 方法2: 使用np.array()
np_from_array = np.array(tensor)
print(f"np.array()结果:\n{np_from_array}")
print(f"类型: {type(np_from_array)}")

## 3. 默认数据类型差异

**关键差异:**
- NumPy默认浮点类型: `float64` (双精度)
- TensorFlow默认浮点类型: `float32` (单精度)

这一差异源于GPU计算优化——`float32`在GPU上的计算速度通常是`float64`的2-8倍，且对于大多数深度学习任务，单精度已经足够。

In [ ]:
# 默认数据类型对比

# NumPy默认float64
np_float_array = np.array([1.0, 2.0, 3.0])
print(f"NumPy默认浮点类型: {np_float_array.dtype}")

# TensorFlow默认float32
tf_float_tensor = tf.constant([1.0, 2.0, 3.0])
print(f"TensorFlow默认浮点类型: {tf_float_tensor.dtype}\n")

# 整数类型对比
np_int_array = np.array([1, 2, 3])
tf_int_tensor = tf.constant([1, 2, 3])
print(f"NumPy默认整数类型: {np_int_array.dtype}")
print(f"TensorFlow默认整数类型: {tf_int_tensor.dtype}")

In [ ]:
# 保持数据类型一致的最佳实践

# 方法1: NumPy创建时指定float32
np_array_f32 = np.array([1.0, 2.0, 3.0], dtype=np.float32)
tensor_f32 = tf.constant(np_array_f32)
print(f"指定NumPy为float32后转换: {tensor_f32.dtype}")

# 方法2: TensorFlow转换时指定dtype
np_array_f64 = np.array([1.0, 2.0, 3.0])  # 默认float64
tensor_converted = tf.constant(np_array_f64, dtype=tf.float32)
print(f"转换时指定dtype: {tensor_converted.dtype}")

# 方法3: 使用tf.cast进行类型转换
tensor_f64 = tf.constant([1.0, 2.0, 3.0], dtype=tf.float64)
tensor_casted = tf.cast(tensor_f64, dtype=tf.float32)
print(f"使用tf.cast转换: {tensor_casted.dtype}")

## 4. 混合运算

TensorFlow操作可以直接接受NumPy数组作为输入，框架会自动进行转换。

In [ ]:
# 混合运算示例

np_array = np.array([[1, 2], [3, 4]], dtype=np.float32)
tf_tensor = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

# TensorFlow操作直接接受NumPy数组
result_add = tf.add(np_array, tf_tensor)
print(f"tf.add(NumPy, Tensor):\n{result_add}\n")

# 运算符也支持混合运算
result_mul = np_array * tf_tensor
print(f"NumPy * Tensor:\n{result_mul}\n")

# 矩阵乘法
result_matmul = tf.matmul(np_array, tf_tensor)
print(f"tf.matmul(NumPy, Tensor):\n{result_matmul}")

## 5. 内存共享与数据复制

理解数据转换时的内存行为对于优化性能至关重要。

**核心规则:**
- CPU上的张量与NumPy数组可能共享内存
- GPU上的张量转换为NumPy时必须复制数据

In [ ]:
# 内存共享验证

# 创建NumPy数组
np_original = np.array([1, 2, 3, 4], dtype=np.float32)
print(f"原始NumPy数组: {np_original}")

# 转换为张量（CPU上可能共享内存）
tensor = tf.constant(np_original)

# 张量转回NumPy
np_converted = tensor.numpy()
print(f"转换后的NumPy数组: {np_converted}")

# 检查是否共享内存（仅在CPU上可能为True）
print(f"\n内存地址比较:")
print(f"原始数组地址: {np_original.__array_interface__['data'][0]}")
print(f"转换数组地址: {np_converted.__array_interface__['data'][0]}")
print(f"是否共享内存: {np.shares_memory(np_original, np_converted)}")

## 6. 性能考量

频繁的类型转换会带来显著的性能开销，特别是在GPU训练场景中。

In [ ]:
# 性能测试: 转换开销

# 创建大型数组
large_np_array = np.random.randn(1000, 1000).astype(np.float32)

# 测试NumPy到TensorFlow的转换时间
n_iterations = 100

start_time = time.time()
for _ in range(n_iterations):
    tensor = tf.constant(large_np_array)
elapsed_np_to_tf = time.time() - start_time

print(f"NumPy -> TensorFlow ({n_iterations}次): {elapsed_np_to_tf:.4f}秒")
print(f"平均每次: {elapsed_np_to_tf/n_iterations*1000:.2f}毫秒\n")

# 测试TensorFlow到NumPy的转换时间
tensor = tf.constant(large_np_array)

start_time = time.time()
for _ in range(n_iterations):
    np_array = tensor.numpy()
elapsed_tf_to_np = time.time() - start_time

print(f"TensorFlow -> NumPy ({n_iterations}次): {elapsed_tf_to_np:.4f}秒")
print(f"平均每次: {elapsed_tf_to_np/n_iterations*1000:.2f}毫秒")

## 7. 最佳实践

### 7.1 数据预处理阶段的建议

In [ ]:
# 最佳实践: 数据预处理

# 反面示例: 每次循环都进行转换（低效）
def inefficient_preprocessing(data_list):
    results = []
    for data in data_list:
        # 每次迭代都进行转换，产生额外开销
        tensor = tf.constant(data)
        processed = tf.nn.relu(tensor)
        results.append(processed.numpy())
    return results

# 正面示例: 批量处理（高效）
def efficient_preprocessing(data_list):
    # 一次性转换所有数据
    batch_tensor = tf.constant(np.array(data_list))
    # 批量处理
    processed = tf.nn.relu(batch_tensor)
    return processed.numpy()

# 性能对比
test_data = [np.random.randn(100, 100).astype(np.float32) for _ in range(50)]

start = time.time()
_ = inefficient_preprocessing(test_data)
print(f"低效方法耗时: {time.time() - start:.4f}秒")

start = time.time()
_ = efficient_preprocessing(test_data)
print(f"高效方法耗时: {time.time() - start:.4f}秒")

### 7.2 使用tf.data构建数据管道

In [ ]:
# 使用tf.data构建高效数据管道

# 模拟训练数据
X_train = np.random.randn(1000, 10).astype(np.float32)
y_train = np.random.randint(0, 2, size=(1000,)).astype(np.float32)

# 创建tf.data.Dataset（推荐方式）
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)  # 异步预取

# 验证数据管道
for batch_x, batch_y in dataset.take(1):
    print(f"批次X形状: {batch_x.shape}")
    print(f"批次X类型: {batch_x.dtype}")
    print(f"批次y形状: {batch_y.shape}")
    print(f"批次y类型: {batch_y.dtype}")

### 7.3 避免在计算图中混用NumPy

In [ ]:
# 在@tf.function中避免使用NumPy操作

# 反面示例: 在tf.function中使用NumPy（会破坏图优化）
@tf.function
def bad_function(x):
    # 下面这行会触发eager执行，破坏图优化
    # numpy_result = np.square(x.numpy())  # 不要这样做！
    pass

# 正面示例: 完全使用TensorFlow操作
@tf.function
def good_function(x):
    return tf.square(x)

# 性能对比
test_tensor = tf.random.normal((1000, 1000))

# 预热
_ = good_function(test_tensor)

# 测试tf.function性能
start = time.time()
for _ in range(100):
    _ = good_function(test_tensor)
print(f"tf.function计算耗时: {time.time() - start:.4f}秒")

## 8. 互操作性功能汇总

In [ ]:
# 常用互操作函数汇总

np_array = np.array([1.0, 2.0, 3.0], dtype=np.float32)
tf_tensor = tf.constant([4.0, 5.0, 6.0])

print("=" * 50)
print("NumPy -> TensorFlow 转换方法")
print("=" * 50)
print(f"tf.constant(np_array): {tf.constant(np_array)}")
print(f"tf.convert_to_tensor(np_array): {tf.convert_to_tensor(np_array)}")

print("\n" + "=" * 50)
print("TensorFlow -> NumPy 转换方法")
print("=" * 50)
print(f"tensor.numpy(): {tf_tensor.numpy()}")
print(f"np.array(tensor): {np.array(tf_tensor)}")

print("\n" + "=" * 50)
print("类型转换")
print("=" * 50)
print(f"tf.cast(tensor, tf.float64): {tf.cast(tf_tensor, tf.float64)}")

## 知识点总结

| 场景 | 推荐做法 | 原因 |
|-----|---------|------|
| 数据准备 | 使用`np.float32` | 与TensorFlow默认类型一致 |
| 数据加载 | 使用`tf.data.Dataset` | 自动优化、异步预取 |
| 模型计算 | 避免`.numpy()`调用 | 保持在GPU上计算 |
| 结果提取 | 训练结束后再转换 | 减少数据传输 |

**关键要点:**
1. NumPy默认`float64`，TensorFlow默认`float32`，需注意类型统一
2. 频繁的类型转换会带来性能开销，应批量处理
3. 在`@tf.function`中应完全使用TensorFlow操作
4. 优先使用`tf.data`管道而非手动转换
5. GPU上的张量转NumPy需要数据复制，应避免在训练循环中使用